### Table 3-4: Directories on Linux Systems

| Directory | Comments |
| :--- | :--- |
| `/` | The root directory where everything begins. |
| `/bin` | Essential binaries for system booting and running. |
| `/boot` | Contains the Linux kernel, RAM disk, and boot loader (GRUB). |
| `/dev` | Device nodes; where the kernel maintains hardware as files. |
| `/etc` | System-wide configuration files (e.g., `fstab`, `passwd`). |
| `/home` | Personal directories for ordinary users. |
| `/lib` | Shared library files for core system programs. |
| `/lost+found` | Used for recovering data after filesystem corruption. |
| `/media` | Automatic mount points for removable media (USB, CD). |
| `/mnt` | Manual mount points for temporary devices. |
| `/opt` | Optional/commercial software installation directory. |
| `/proc` | Virtual filesystem providing a window into the kernel. |
| `/root` | Home directory for the superuser (root). |
| `/run` | Modern tempfs replacement for `/tmp` stored in RAM. |
| `/sbin` | System binaries reserved for the superuser. |
| `/sys` | Detailed hardware information detected by the kernel. |
| `/tmp` | Temporary, transient files; often cleared on reboot. |
| `/usr` | Largest tree; contains user programs and support files. |
| `/usr/bin` | Executable programs installed by the distribution. |
| `/usr/lib` | Shared libraries for programs in `/usr/bin`. |
| `/usr/local` | Programs installed manually (e.g., compiled from source). |
| `/usr/sbin` | Advanced system administration programs. |
| `/usr/share` | Shared data (icons, documentation, backgrounds). |
| `/usr/share/doc` | Documentation files organized by package. |
| `/var` | Variable data: databases, mail, and print spools. |
| `/var/log` | Records of system activity and error logs. |
| `~/.config` | User-specific application configuration files. |


## 🛠️ Permissions & User Management Cheatsheet

---
# My Diary

Today  **{{DATE_PRETTY}}**.

## General Counter
It has been    **{{DAY_SINCE_2025_08_18}}** days since I started this diary.

# Repository Counter

Day **{{DAYS_SINCE_REPO_START}}** From I started this repository

## SDN Counter 
This is the day  **{{DAYS_SINCE_FOLDER_START}}** since I started my notes on Master LInux 


#Linux Kernel Mastery: System I/O, Security & Process Architecture



---

## 1. The Philosophy of Multi-User I/O: The Kernel as Gatekeeper

Linux is not merely a "multitasking" system; it is fundamentally a **multiuser** system deeply rooted in the Unix tradition [1]. This distinction is critical. In the days of centralized mainframes, the OS had to ensure that one user could not crash the system or spy on another user's data [2, 3].

Consequently, the Linux Kernel acts as the ultimate gatekeeper. Every Input/Output (I/O) operation—whether reading a config file or writing to a socket—is checked against a strict permission matrix.

### The Mathematics of `rwx`: Bitmasks & Octal Logic

To a novice, `rwx` (Read, Write, Execute) are just letters. To the Kernel, they are **bits**.

Computer logic is binary (base 2). Permissions are grouped into sets of three (User, Group, Other). Three bits of binary data ($2^3$) map perfectly to a single digit in the **Octal** (base 8) number system [4].

#### The Bitmask Map
When the Kernel checks permissions, it performs a bitwise operation.

| Attribute | Binary | Octal Value | Meaning |
| :--- | :--- | :--- | :--- |
| **Read (r)** | `100` | **4** | High bit set. |
| **Write (w)** | `010` | **2** | Middle bit set. |
| **Execute (x)** | `001` | **1** | Low bit set. |
| **None (-)** | `000` | **0** | No bits set. |

**Why Octal?**
Because it is human-readable shorthand for the machine's binary reality.
*   **Example:** `rwx` = `111` (Binary) = $4+2+1$ = **7** (Octal).
*   **Example:** `r-x` = `101` (Binary) = $4+0+1$ = **5** (Octal) [5].

> **Linus Level Insight:**
> Never guess at permissions. If you see `chmod 777`, you are looking at laziness that opens security holes. A true engineer calculates the **exact** mask needed. If a file only needs to be read by the group, the mask is `040` (binary `000 100 000`). Understanding the binary mapping allows you to visualize the exact switches you are flipping in the inode.

### The Trap of `umask`
The `umask` (User Mask) is a filter. It does not set permissions; it **removes** them from the system default (usually `666` for files, `777` for directories) [6].

*   **Logic:** `Result = Default AND (NOT Mask)`
*   **Scenario:** If `umask` is `0002` (binary `...000 010`), it explicitly *turns off* the write bit for "others" [7].

> **Gotcha:** A common mistake is thinking `umask 000` gives no permissions. It actually subtracts *nothing*, resulting in a file that is world-writable [8]. This is rarely what you want in a secure environment.

---

## 2. Identity & Security Architecture

In the Kernel's eyes, you are not a username; you are a **User ID (uid)** and a **Group ID (gid)** [9].

### The Privilege Model: `su` vs. `sudo`

The transition from a regular user to a Superuser (uid 0) is the most critical security boundary in the OS.

1.  **`su` (Substitute User):**
    *   **Mechanism:** Spawns a new shell with the target user's ID [10].
    *   **The Flaw:** Requires sharing the root password. In a team of 10 sysadmins, if one leaves, you must change the root password. It encourages logging in as root, which is dangerous [11].
2.  **`sudo` (SuperUser DO):**
    *   **Mechanism:** Grants temporary privilege escalation based on a config file (`/etc/sudoers`).
    *   **The Advantage:** Users authenticate with *their own* password [12].
    *   **Audit Trail:** It logs *who* did *what*. This adheres to the **Principle of Least Privilege**—granting superuser powers only for the specific command needed, not for an entire shell session [11].

> **Linus Level Insight:**
> "Permission Denied" is not an error; it is a feature. It is the system protecting itself from you. Modern distributions like Ubuntu disable the root account password entirely by default to force the use of `sudo` [13]. This prevents the catastrophic habit of "operating as root" for daily tasks, which makes Linux as vulnerable as Windows 95.

### Advanced Permission Bits: The Dangerous Trio

Beyond `rwx`, there are three special bits used for architectural control.

| Bit Name | Octal | Symbol | Function | Danger/Use Case |
| :--- | :--- | :--- | :--- | :--- |
| **Setuid** | **4000** | `rws...` | Executable runs with **Owner's** privileges, not the user's [14]. | **CRITICAL:** If a program owned by root has this set and has a bug, a standard user can exploit it to become root. |
| **Setgid** | **2000** | `...rws...` | Files created in directory inherit the **Directory's** group, not the user's primary group [15]. | Essential for shared collaboration directories (e.g., `/var/www` or shared music folders) [16]. |
| **Sticky** | **1000** | `...rwt` | On directories: Only the **Owner** (or root) can delete a file, regardless of write permissions [17]. | Prevents users from deleting each other's temporary files in shared spaces like `/tmp`. |

---

## 3. Process Management: The Engine

### The Process Abstraction
A process is the kernel's method of "organizing the different programs waiting for their turn at the CPU" [18]. It provides an illusion of simultaneity.

From a kernel perspective, a process provides **Isolation**.
*   It has its own memory space.
*   It has its own user context (uid/gid).
*   One user's crashed process cannot corrupt the memory of another user's process [3].

### Signals as Inter-Process Communication (IPC)
When you run `kill`, you are not just "stopping" a program. You are sending a **Signal**—a primitive form of IPC [19].

*   `kill 1234` sends a signal (default `SIGTERM`) to process ID 1234.
*   The process receives this signal and decides how to handle it (e.g., save state and exit clean).
*   **Architecture Note:** This is why "zombie" processes exist; they are processes that have finished execution but are waiting for the parent process to acknowledge their exit signal.

> **Linus Level Insight:**
> `kill -9` (SIGKILL) should be your last resort. It doesn't tell the process to stop; it tells the Kernel to rip the process out of memory immediately. The process gets no chance to clean up open file descriptors or database connections. A master administrator always tries `SIGTERM` (standard `kill`) first to allow for a graceful shutdown.

---

## Summary Cheat Sheet

*   **`chmod`:** 4 (Read) + 2 (Write) + 1 (Execute).
*   **`chown user:group`:** Changes ownership logic [20].
*   **`sudo`:** Defines *authorization* (who can do what) without sharing *authentication* (root password) [12, 21].
*   **Directory `x` bit:** Allows you to **enter** the directory, not run it [22]. Without `x` on a folder, you cannot `cd` into it even if you have read permissions.



## APPENDIX 

| Command | Description | Usage Context |
| :--- | :--- | :--- |
| **`id`** | Display user identity (UID, GID). | verifying who you are currently logged in as. |
| **`chmod`** | Change a file’s mode. | Modifying read/write/execute permissions. |
| **`umask`** | Set the default file permissions. | Security baseline for new files. |
| **`su`** | Run a shell as another user. | Switching to root or another user account completely. |
| **`sudo`** | Execute a command as another user. | Running a single administrative command. |
| **`chown`** | Change a file’s owner. | Transferring file ownership to a different user. |
| **`chgrp`** | Change a file’s group ownership. | Changing which group controls the file. |
| **`addgroup`**| Add a user or a group to the system. | Creating new groups for permission management. |
| **`usermod`** | Modify a user account. | Changing user groups, shell, or home directory. |
| **`passwd`** | Change a user’s password. | Updating credentials for security. |



| Attribute | File type |
| :---: | :--- |
| `-` | A regular file. |
| `d` | A directory. |
| `l` | A symbolic link. Notice that with symbolic links, the remaining file attributes are always `rwxrwxrwx` and are dummy values. The real file attributes are those of the file the symbolic link points to. |
| `c` | A character special file. This file type refers to a device that handles data as a stream of bytes, such as a terminal or `/dev/null`. |



| Attribute | Files | Directories |
| :---: | :--- | :--- |
| `r` | Allows a file to be opened and read. | Allows a directory's contents to be listed, but no file information is available unless the execute attribute is also set. |
| `w` | Allows a file to be written to or truncated; however, this attribute does not allow files to be renamed or deleted. The ability to delete or rename files is determined by directory attributes. | Allows files within a directory to be created, deleted, and renamed if the execute attribute is also set. |
| `x` | Allows a file to be treated as a program and executed. Program files written in scripting languages must also be set as readable to be executed. | Allows a directory to be entered (that is, `cd directory`) and directory metadata (that is, `ls -l directory`) to be accessed. File operations such as `cp`, `rm`, and `mv` require this access to the directory. |